<a href="https://colab.research.google.com/github/thien2001git/HoangXuanThienDoAnTotNghiep/blob/main/AudioFilter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow-io

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import math
import tensorflow as tf
import pandas as pd
import tensorflow_io as tfio
from keras import layers
from keras import models
import os
import librosa
import pickle
from IPython.display import Audio
from IPython.core.display import display
import random
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Conv1D, MaxPooling1D, Dropout


In [3]:
def getAllFile(root):
    l = []
    for path, subdirs, files in os.walk(root):
        for name in files:
            l.append(os.path.join(path, name))
    return l
def dumpToFile(obj, path):
    with open(path, 'wb') as file:
        pickle.dump(obj, file)
def loadToObj(path):
    with open(path, 'rb') as file:
        return pickle.load(file)


In [4]:
path = ["drive/MyDrive/distant_16k.bin",
        "drive/MyDrive/source_16k_train.bin",
        'drive/MyDrive/speech_train.bin',
        "drive/MyDrive/source_16k_test.bin",
        "drive/MyDrive/speech_test.bin", ]
# distant_16k = loadToObj(path[0])
source_16k_train = loadToObj(path[1])
speech_train = loadToObj(path[2])
source_16k_test = loadToObj(path[3])
speech_test = loadToObj(path[4])

In [5]:
train = pd.read_csv("/content/drive/MyDrive/VOiCES_devkit/references/train_index.csv")
test = pd.read_csv("/content/drive/MyDrive/VOiCES_devkit/references/test_index.csv")

In [6]:
SR = 16000

In [7]:
def toSound(e):
    file = os.path.join("/content/drive/MyDrive/VOiCES_devkit", e)
    
    y, sr = librosa.load(file, sr = 16000)
    y = y[:16000*18]
    zr = np.zeros((16000*18 - y.shape[0]))
    y = np.append(y, zr)
    audio_slices = tf.keras.utils.timeseries_dataset_from_array(y,y,sequence_length=48000,sequence_stride=48000,batch_size=1)
    l = []

    for i in audio_slices:
        e1 = i[0].numpy()[0]
        l.append(e1)
        # spectrogram = librosa.feature.melspectrogram(y=e1, sr=sr)
        # spectrogram = librosa.power_to_db(spectrogram, ref=np.max)
        # spectrogram = (spectrogram - spectrogram.min()) / (spectrogram.max() - spectrogram.min())
        # l.append(spectrogram)
    return l

In [8]:
X_train = []
y_train = []

for i in train['filename'][:200]:
    X_train.extend(toSound(i))
for i in train['source'][:200]:
    y_train.extend(toSound(i))

In [9]:
X_test = []
y_test = []

for i in test['filename'][:20]:
    X_test.extend(toSound(i))
for i in test['source'][:20]:
    y_test.extend(toSound(i))

In [10]:
len(X_test)

120

In [11]:
hop_length = int(math.pow(2,10))
def showSome(y, sr):
    
    # librosa.display.waveshow(y)
    # plt.show()
    
    img = librosa.display.specshow(y, y_axis='log', sr=sr, hop_length=hop_length,
                            x_axis='time')

In [12]:
y_train[0].shape

(48000,)

In [13]:
input_shape=(48000, 1)
from keras.layers import Input, Conv1D, Conv1DTranspose
from keras.models import Model

def create_audio_filter_model(input_shape):
    # Định nghĩa đầu vào
    input_audio = Input(shape=input_shape, name='input_audio')

    # Phần encoder
    x = Conv1D(64, 3, activation='relu', padding='same')(input_audio)
    x = Conv1D(128, 3, activation='relu', padding='same')(x)
    x = Conv1D(256, 3, activation='relu', padding='same')(x)

    # Phần decoder
    # x = Conv1DTranspose(256, 3, strides=1, activation='relu', padding='same')(encoded)
    # x = Conv1DTranspose(64, 3, strides=1, activation='relu', padding='same')(x)
    decoded = Conv1D(1, 3, activation='linear', padding='same')(x)

    # Định nghĩa mô hình
    model = Model(inputs=input_audio, outputs=decoded, name='audio_filter_model')

    # Compile mô hình
    model.compile(optimizer='adam', loss='mse')

    return model


model = create_audio_filter_model(input_shape)
model.summary()

Model: "audio_filter_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_audio (InputLayer)    [(None, 48000, 1)]        0         
                                                                 
 conv1d (Conv1D)             (None, 48000, 64)         256       
                                                                 
 conv1d_1 (Conv1D)           (None, 48000, 128)        24704     
                                                                 
 conv1d_2 (Conv1D)           (None, 48000, 256)        98560     
                                                                 
 conv1d_3 (Conv1D)           (None, 48000, 1)          769       
                                                                 
Total params: 124,289
Trainable params: 124,289
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(np.array(X_train),np.array(y_train),batch_size=50,epochs=10,validation_data=(np.array(X_test),np.array(y_test)), callbacks=tf.keras.callbacks.EarlyStopping(verbose=1, 
                                               patience=5,
                                               restore_best_weights=True
                                              ), )


Epoch 1/10
16/24 [===================>..........] - ETA: 6:24 - loss: 0.0111

In [ ]:
metrics = history.history
plt.plot(history.epoch, metrics['val_mean_absolute_error'])
plt.legend(['val_mean_absolute_error'])
plt.show()

In [ ]:
plt.plot(history.epoch, metrics['val_loss'])
plt.legend(['val_loss'])
plt.show()

In [ ]:
v = model.predict(np.array(X_test[:1]))

In [ ]:
v